<a href="https://colab.research.google.com/github/28paranoid/non-communicable-deseases/blob/main/01_cleaned_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from google.colab import files
import pandas as pd

# Upload CSV file
uploaded = files.upload()

# Automatically get the uploaded filename
filename = list(uploaded.keys())[0]

# Load CSV into a DataFrame
df = pd.read_csv(filename)

# Show first 5 rows to check
df.head()

Saving noncommunicable_diseases_indicators_btn.csv to noncommunicable_diseases_indicators_btn.csv


,GHO (CODE),GHO (DISPLAY),GHO (URL),YEAR (DISPLAY),STARTYEAR,ENDYEAR,REGION (CODE),REGION (DISPLAY),COUNTRY (CODE),COUNTRY (DISPLAY),DIMENSION (TYPE),DIMENSION (CODE),DIMENSION (NAME),Numeric,Value,Low,High
0,#indicator+code,#indicator+name,#indicator+url,#date+year,#date+year+start,#date+year+end,#region+code,#region+name,#country+code,#country+name,#dimension+type,#dimension+code,#dimension+name,#indicator+value+num,#indicator+value,#indicator+value+low,#indicator+value+high
1,NCD_CHOL_MEANNONHDL_A,"Mean Non-HDL cholesterol, age-standardized",https://www.who.int/data/gho/data/indicators/i...,1991,1991,1991,SEAR,South-East Asia,BTN,Bhutan,SEX,SEX_FMLE,Female,3.0,3.0 [2.7-3.3],2.7,3.3
2,SA_0000001400,"Alcohol, recorded per capita (15+) consumption...",https://www.who.int/data/gho/data/indicators/i...,1989,1989,1989,SEAR,South-East Asia,BTN,Bhutan,ALCOHOLTYPE,ALCOHOLTYPE_SA_BEER,Beer,0.31,0.31,NaN,NaN
3,SA_0000001747,"Alcohol, recorded per capita (15+) consumption...",https://www.who.int/data/gho/data/indicators/i...,2020,2020,2020,SEAR,South-East Asia,BTN,Bhutan,NaN,NaN,NaN,2.719382025,2.7 [1.9 - 3.5],1.90229731,3.542702365
4,NCD_BMI_PLUS1C,Prevalence of overweight among children and ad...,https://www.who.int/data/gho/data/indicators/i...,2010,2010,2010,SEAR,South-East Asia,BTN,Bhutan,SEX,SEX_MLE,Male,9.0024188,9.0 [3.5-18.0],3.46319,18.038097


In [8]:
import numpy as np

# 1️⃣ Drop first row (metadata)
df = df.drop(0).reset_index(drop=True)

# 2️⃣ Clean column names
df.columns = (
    df.columns
      .str.strip()
      .str.replace(r"[#,+()]", "", regex=True)
      .str.replace(" ", "_")
)

# 3️⃣ Convert year columns to numeric
for col in ["YEAR_DISPLAY", "STARTYEAR", "ENDYEAR"]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce").astype("Int64")

# 4️⃣ Extract numeric part from 'Value' column
def extract_numeric(value):
    if isinstance(value, str):
        first_part = value.split()[0]
        try:
            return float(first_part)
        except:
            return np.nan
    return value

if "Value" in df.columns:
    df["Value_num"] = df["Value"].apply(extract_numeric)

# 5️⃣ Convert numeric columns to floats
for col in ["Numeric", "Low", "High", "Value_num"]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

# 6️⃣ Drop fully empty columns
df = df.dropna(axis=1, how="all")

# 7️⃣ Replace blank strings with NaN
df = df.replace("", np.nan)

# 8️⃣ Keep only rows with actual Value_num
df = df[df["Value_num"].notna()].reset_index(drop=True)

# 9️⃣ Keep only important columns
columns_to_keep = ["COUNTRY_DISPLAY", "YEAR_DISPLAY", "STARTYEAR", "ENDYEAR", "GHO_DISPLAY", "DIMENSION_NAME", "Value_num"]
df_cleaned = df[columns_to_keep].copy()

# 1️⃣0️⃣ Check cleaned data
df_cleaned.head(), df_cleaned.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3714 entries, 0 to 3713
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   COUNTRY_DISPLAY  3714 non-null   object 
 1   YEAR_DISPLAY     3714 non-null   Int64  
 2   STARTYEAR        3714 non-null   Int64  
 3   ENDYEAR          3714 non-null   Int64  
 4   GHO_DISPLAY      3714 non-null   object 
 5   DIMENSION_NAME   3659 non-null   object 
 6   Value_num        3714 non-null   float64
dtypes: Int64(3), float64(1), object(3)
memory usage: 214.1+ KB


(  COUNTRY_DISPLAY  YEAR_DISPLAY  STARTYEAR  ENDYEAR  \
 0          Bhutan          1991       1991     1991   
 1          Bhutan          1989       1989     1989   
 2          Bhutan          2020       2020     2020   
 3          Bhutan          2010       2010     2010   
 4          Bhutan          2012       2012     2012   
 
                                          GHO_DISPLAY DIMENSION_NAME  Value_num  
 0         Mean Non-HDL cholesterol, age-standardized         Female       3.00  
 1  Alcohol, recorded per capita (15+) consumption...           Beer       0.31  
 2  Alcohol, recorded per capita (15+) consumption...            NaN       2.70  
 3  Prevalence of overweight among children and ad...           Male       9.00  
 4  Premature deaths due to noncommunicable diseas...           Male      49.00  ,
 None)

In [9]:
df_cleaned.to_csv("cleaned_data.csv", index=False)
from google.colab import files
files.download("cleaned_data.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Task
Create a machine learning model to predict a suitable variable from the `df_cleaned` dataset and then build a Streamlit dashboard to visualize the data and integrate the trained model for making predictions.

## Identify Prediction Goal

### Subtask:
Determine the specific variable or outcome you wish to predict from the `df_cleaned` dataset. This will guide the choice of machine learning model.


**Reasoning**:
To identify a prediction goal, I need to examine the `GHO_DISPLAY` and `Value_num` columns. First, I will display descriptive statistics for both columns to understand their distributions and unique values.



In [10]:
print(df_cleaned['GHO_DISPLAY'].value_counts())
print(df_cleaned['Value_num'].describe())

GHO_DISPLAY
Prevalence of overweight among children and adolescents, BMI > +1 standard deviations above the median (crude estimate) (%)                      297
Prevalence of thinness among children and adolescents, BMI < -2 standard deviations below the median (crude estimate) (%)                        297
Prevalence of obesity among children and adolescents, BMI > +2 standard deviations above the median (crude estimate) (%)                         297
Alcohol, recorded per capita (15+) consumption (in litres of pure alcohol), by beverage type                                                     281
Prevalence of diabetes, age-standardized                                                                                                         198
Prevalence of diabetes, crude                                                                                                                    198
Mean HDL cholesterol, age-standardized                                                        

The prediction goal is to predict the `Value_num` for the indicator: **'Prevalence of overweight among children and adolescents, BMI > +1 standard deviations above the median (crude estimate) (%)'**. This will be a regression problem, as the target variable (`Value_num`) is continuous.

## Develop Machine Learning Model

### Subtask:
Prepare the data for model training by filtering for the target indicator, defining features and target, and preprocessing the data.


**Reasoning**:
To prepare the data for model training, I will filter the dataframe, define target and features, handle missing values, apply one-hot encoding, and split the data into training and testing sets as per the instructions.



In [11]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

# 1. Filter the df_cleaned DataFrame for the target indicator
df_model = df_cleaned[df_cleaned['GHO_DISPLAY'] == 'Prevalence of overweight among children and adolescents, BMI > +1 standard deviations above the median (crude estimate) (%)'].copy()

# 2. Define the target variable y
y = df_model['Value_num']

# 3. Identify suitable features for predicting Value_num
X = df_model[['YEAR_DISPLAY', 'DIMENSION_NAME']].copy()

# 4. Handle any missing values in the selected features
X['DIMENSION_NAME'] = X['DIMENSION_NAME'].fillna('Unknown')

# 5. Apply one-hot encoding to the 'DIMENSION_NAME' categorical feature
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
dimension_encoded = encoder.fit_transform(X[['DIMENSION_NAME']])
dimension_df = pd.DataFrame(dimension_encoded, columns=encoder.get_feature_names_out(['DIMENSION_NAME']), index=X.index)

# Drop original categorical column and concatenate encoded features
X = pd.concat([X.drop('DIMENSION_NAME', axis=1), dimension_df], axis=1)

# 6. Split the preprocessed data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Data preparation complete.")
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

Data preparation complete.
X_train shape: (237, 4)
X_test shape: (60, 4)
y_train shape: (237,)
y_test shape: (60,)


## Train and Evaluate Regression Model

### Subtask:
Train a regression model using the preprocessed training data and evaluate its performance on the test set.


**Reasoning**:
To train and evaluate the regression model, I will import the necessary libraries, instantiate a RandomForestRegressor, train it with the provided training data, make predictions on the test data, and then calculate and print the R-squared, MAE, and MSE scores.



In [12]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# 1. Instantiate a RandomForestRegressor model
model = RandomForestRegressor(random_state=42)

# 2. Train the model
model.fit(X_train, y_train)

# 3. Make predictions on the X_test data
y_pred = model.predict(X_test)

# 4. Calculate and print the evaluation metrics
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print(f"Model Evaluation:\n")
print(f"R-squared: {r2:.3f}")
print(f"Mean Absolute Error (MAE): {mae:.3f}")
print(f"Mean Squared Error (MSE): {mse:.3f}")

Model Evaluation:

R-squared: 0.983
Mean Absolute Error (MAE): 0.439
Mean Squared Error (MSE): 0.385


## Build Streamlit Dashboard

### Subtask:
Create an interactive Streamlit application to visualize the data and integrate the trained model for making predictions.


## Summary:

### Q&A
The suitable variable chosen for prediction is `Value_num` for the indicator: 'Prevalence of overweight among children and adolescents, BMI > +1 standard deviations above the median (crude estimate) (%)'. This is a regression problem because `Value_num` is a continuous variable.

### Data Analysis Key Findings
*   The target indicator, 'Prevalence of overweight among children and adolescents, BMI > +1 standard deviations above the median (crude estimate) (%)', was selected for prediction, as it had 297 entries in the `df_cleaned` dataset.
*   The `Value_num` variable, chosen as the target, is continuous with a mean of approximately 38.18 and ranges from 0 to 2880.
*   A `RandomForestRegressor` model was trained using 'YEAR\_DISPLAY' and one-hot encoded 'DIMENSION\_NAME' as features.
*   The model achieved strong performance metrics on the test set: an R-squared of 0.983, a Mean Absolute Error (MAE) of 0.439, and a Mean Squared Error (MSE) of 0.385.

### Insights or Next Steps
*   The machine learning model demonstrates excellent predictive capability for the prevalence of overweight in children and adolescents, suggesting it can be reliably used for estimating this indicator.
*   The next step is to integrate this trained model into an interactive Streamlit dashboard, as outlined in the initial task, to allow for visualization and user-driven predictions.
